## Programowanie dynamiczne – liniowe zagadnienie załadunku
### Szymon Wysogląd
W ramach ćwiczenia zostały wykonane obydwa algorytmy:
* Binarnego problemu załadunku 0-1 KP
* Całkowitoliczbowego problemu liniowego

In [1]:
import numpy as np
import pandas as pd
from typing import List
import random

### Zadanie 1
#### Całkowitoliczbowy problem liniowy

In [2]:
class Item:
  def __init__(self, a, d, q):
    self.weight = a
    self.count = d
    self.function = q
    
  def maxCount(self, y):
    r = y // self.weight
    return r if r < self.count else self.count
  
  def __str__(self):
    return f'{self.weight}:{self.count}'
  
  def __repr__(self):
    return self.__str__()

In [3]:
class Ship:
  def __init__(self, capacity):
    self.capacity = capacity
    self.items: "Item" = []
    self.m = None
    self.l = None
    
  def addItem(self, item):
    self.items.append(item)
    
  def __getOptimalMatrix(self):
    #m -> macierz, ktora w kazdym polu zawiera krotke (l. elementow, wartosc straty)
    m = np.empty((self.capacity + 1, len(self.items)), dtype=object)
    # first column
    item = self.items[0]
    for x in range(self.capacity + 1):
      count = item.maxCount(x)
      m[x][0] = (count, item.function(count))
    
    for x in range(1, len(self.items)):
      for y in range(self.capacity + 1):
        # count określa ile przedmiotów możemy wziąć
        item = self.items[x]
        count = item.maxCount(y)
        
        minF = m[y, x-1][1] + item.function(0)
        bestCount = 0
        
        #szukanie minimum biorąc różną liczbę danego towaru 
        for i in range(count + 1):
          # f to zmienna, która określa "karę", gdy przewieziemy i sztuk aktualnego produktu, 
          # oraz reszte maksymalna z poprzedniego ruchu
          f = item.function(i) + m[y - item.weight * i, x-1][1]
          if f < minF:
            bestCount = i
            minF =f
        m[y,x] = (bestCount, minF)
    return m
  
  def __getCountEachItem(self, m):
    y = self.capacity
    l = []
    n = len(self.items)
    for x in range(n):
      #zwracamy ilość w tabeli
      l.append(m[y,n-x-1][0])
      y -= self.items[n-x-1].weight * l[-1]
    return l[::-1] 
    
    
  def solve(self):
    m = self.__getOptimalMatrix()
    res = self.__getCountEachItem(m)
    self.m = m
    self.l = res
    return m, res
    
  def printData(self):
    print("\nTabela danych:")
    rowheaders = ["waga", "potrzebne sztuki", "funkcja straty"]
    colheaders = [f'x{i}' for i in range(len(self.items))]
    data = np.empty((3, len(self.items)), dtype=object)
    for i, item in enumerate(self.items):
      data[0, i] = item.weight
      data[1, i] = item.count
      data[2, i] = f'{item.function(0)}-{item.function(0) - item.function(1) if item.function(0) - item.function(1) > 1 else "" }x'
    df = pd.DataFrame(data, index=rowheaders, columns=colheaders)
    print(df)
    print('\nCel: Minimalizacja łącznej straty.\n')
    
  def printSolution(self):
    if self.m is None or self.l is None:
      self.solve()
    print("\nTabela wynikowa:")
    rowHeaders = [i for i in range(self.capacity + 1)]
    colHeaders = [f'x{i}' for i in range(len(self.items))]
    df = pd.DataFrame(self.m[:,:], index=rowHeaders, columns=colHeaders) 
    print(df)
    print(f'\nZatem optymalne rozwiązanie to: {self.l} ze stratą {self.m[-1,-1][1]}.')
    print('\n')      
    
  def __str__(self):
    return f'{[str(x) for x in self.items]}'
  
  def __repr__(self):
    return self.__str__()


#### Binarny problem załadunku 0-1 KP

In [4]:
class KnapsackItem:
  def __init__(self, weight, value):
    self.weight = weight
    self.value = value
  
  def __str__(self):
    return f'{self.weight}->{self.value}'
  
  def __repr__(self):
    return self.__str__()

In [5]:
class Knapsack:
  def __init__(self, capacity):
    self.capacity = capacity
    self.items : List["KnapsackItem"] = []
    self.m = None
    self.l = None
    
  def add(self, item : "KnapsackItem"):
    self.items.append(item)
    
  def __getOptimizedMatrix(self):
    m = np.zeros((self.capacity + 1, len(self.items)))
    item = self.items[0]
    # dla pierwszego przedmiotu bierzemy go, jeżeli waga na to pozwala
    m[:, 0] = [item.value if item.weight <= i else 0 for i in range(self.capacity + 1)]
    
    #pętlą przechodzimy po wszystkich przedmiotach po kolei i sprawdzamy
    #jak się sprawy będą miały dla odpowiednich pojemności
    for x in range(1, len(self.items)):
      for y in range(self.capacity + 1):
        item = self.items[x]
        # jeżeli przedmiotu nie możemy wziąć,
        #to przepisujemy wartość dla tej pojemności bez uwzględnienia tego przedmiotu
        if item.weight > y:
          m[y, x] = m[y, x-1]
        else:
          #jeżeli przedmiot się zmieści, to liczymy maksium z:
          #zawartosci plecaka bez tego przedmiotu,
          # tego przedmiotu i reszty, która się wpasuje w pozostałe miejsce
          m[y,x] = max(item.value + m[y - item.weight,x-1], m[y,x-1])
    # zwracamy macierz, której element m[-1,-1] to maksimum problemu
    return m
  
  def getSolution(self):
    m = self.__getOptimizedMatrix()
    l = [0] * len(self.items)
    y, x = m.shape
    y -=1 
    x-=1
    while True:
      if m[y,x] != m[y, x-1] or y > 0 and x == 0:
        l[x] = 1
        y -= self.items[x].weight
      x -= 1
      if y == 0 or x < 0:
        break
    self.m = m
    self.l = l
    return m, l
  
  def printData(self):
    res1 = f'{self.items[0].value}x0'
    res2 = f'{self.items[0].weight}x0'
    
    for i, item in enumerate(self.items):
      if i == 0:
        continue
      res1 += f' + {item.value}x{i}'
      res2 += f' + {item.weight}x{i}'
    print(f'\nFunkcja celu: {res1} -> max')
    print(f'Ograniczenia: {res2} <= {self.capacity}')
    
  def printSolution(self):
    if self.m is None or self.l is None:
      self.getSolution()
    print("\nTabela wynikowa:")
    rowHeaders = [i for i in range(self.capacity + 1)]
    colHeaders = [f'x{i}' for i in range(len(self.items))]
    df = pd.DataFrame(self.m[:,:], index=rowHeaders, columns=colHeaders) 
    print(df)
    print(f'\nZatem optymalne rozwiązanie to: {self.l} z wartością {self.m[-1,-1]}.')
    print('\n')

In [6]:
def knapsackLectureTest():
  data = [(1,1), (4,3), (3,2), (3,2)]
  cap = 7
  b = Knapsack(cap)
  for w, v in data:
    b.add(KnapsackItem(w,v))
  b.printData()
  b.getSolution()
  b.printSolution()
knapsackLectureTest()


Funkcja celu: 1x0 + 3x1 + 2x2 + 2x3 -> max
Ograniczenia: 1x0 + 4x1 + 3x2 + 3x3 <= 7

Tabela wynikowa:
    x0   x1   x2   x3
0  0.0  0.0  0.0  0.0
1  1.0  1.0  1.0  1.0
2  1.0  1.0  1.0  1.0
3  1.0  1.0  2.0  2.0
4  1.0  3.0  3.0  3.0
5  1.0  4.0  4.0  4.0
6  1.0  4.0  4.0  4.0
7  1.0  4.0  5.0  5.0

Zatem optymalne rozwiązanie to: [0, 1, 1, 0] z wartością 5.0.




In [7]:
def createFunc(d, los):
  return lambda x : (d - x) * los if d > x else 0

### Zadanie 2
#### Test metody całkowitoliczbowego problemu liniowego

In [8]:
def test1():
  items = [(3,6,2), (2,5,4), (3,3,3)] 
  capacity = 13
  s = Ship(capacity)
  for a,d,los in items:
    f = createFunc(d, los)
    x = Item(a, d, f)
    s.addItem(x)
  s.printData()
  s.solve()
  s.printSolution()

In [9]:
def test2():
  random.seed(42)
  nrOfItems = 10
  mini = 1
  maxi = 10
  capacity = 20
  items = [(random.randint(mini, maxi), random.randint(mini, maxi), random.randint(mini,maxi // 2)) for _ in range(nrOfItems)]
  s = Ship(capacity)
  for a,d,los in items:
    f = createFunc(d, los)
    x = Item(a, d, f)
    s.addItem(x)
  s.printData()
  s.solve()
  s.printSolution()

In [10]:
test2()


Tabela danych:
                    x0    x1   x2   x3    x4     x5     x6     x7     x8  \
waga                 2     4    2   10     1      4      1      9      8   
potrzebne sztuki     1     4    9    7     2      9      9      7     10   
funkcja straty    3-3x  8-2x  9-x  7-x  4-2x  45-5x  18-2x  14-2x  30-3x   

                     x9  
waga                  1  
potrzebne sztuki      3  
funkcja straty    12-4x  

Cel: Minimalizacja łącznej straty.


Tabela wynikowa:
        x0       x1       x2       x3       x4       x5       x6        x7  \
0   (0, 3)  (0, 11)  (0, 20)  (0, 27)  (0, 31)  (0, 76)  (0, 94)  (0, 108)   
1   (0, 3)  (0, 11)  (0, 20)  (0, 27)  (1, 29)  (0, 74)  (0, 92)  (0, 106)   
2   (1, 0)   (0, 8)  (0, 17)  (0, 24)  (2, 27)  (0, 72)  (0, 90)  (0, 104)   
3   (1, 0)   (0, 8)  (0, 17)  (0, 24)  (1, 26)  (0, 71)  (1, 88)  (0, 102)   
4   (1, 0)   (0, 8)  (1, 16)  (0, 23)  (2, 24)  (0, 69)  (2, 86)  (0, 100)   
5   (1, 0)   (0, 8)  (1, 16)  (0, 23)  (2, 24)  (0, 

### Zadanie 3

* Jakie zakłada się założenia odnośnie wartości wag i zysków przedmiotów?

  Zakłada się, że wartości wag i zysków przedmiotów są liczbami naturalnymi.


* Co się stanie jeśli te założenia nie spełnimy (jaka modyfikacja sposobu
rozwiązania zadania)?

  Możliwe jest przeskalowanie zysków przedmiotów na liczby naturalne, najpierw dodając określoną wartość, a na koniec odejmując ją po znalezieniu rozwiązania.


* Jaka jest złożoność obliczeniowa algorytmu?

  $$O(m \cdot n)$$
  gdzie m i n to wymiary macierzy, czyli liczba zadań i pojemność.

### Źródła
Kod jest opracowaniem własnym na bazie materiałów z wykładu i filmów z YouTube.